# Импорты

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import json

# Чтение исходных данных

In [2]:
DATAPATH = './data/'

In [3]:
train_file = DATAPATH + 'train.json'
test_file = DATAPATH + 'test.json'

In [4]:
sample = pd.read_csv(DATAPATH + 'sample.csv')
sample.head()

,id,sentiment
0,0,neutral
1,1,positive
2,2,negative
3,3,positive
4,4,neutral


In [5]:
def get_df(json_filename):
    with open(json_filename) as json_file:
        temp = json.load(json_file)
        return pd.DataFrame.from_records(temp)

In [6]:
train = get_df(train_file)
train.head()

,text,id,sentiment
0,Досудебное расследование по факту покупки ЕНПФ...,1945,negative
1,Медики рассказали о состоянии пострадавшего му...,1957,negative
2,"Прошел почти год, как железнодорожным оператор...",1969,negative
3,По итогам 12 месяцев 2016 года на территории р...,1973,negative
4,Астана. 21 ноября. Kazakhstan Today - Агентств...,1975,negative


In [7]:
test = get_df(test_file)
test.head()

,text,id
0,"Как сообщает пресс-служба акимата Алматы, для ...",0
1,Казахстанские авиакомпании перевозят 250 тысяч...,1
2,На состоявшемся под председательством Касым-Жо...,2
3,В ОАЭ состоялись переговоры между казахстанско...,3
4,12 вагонов грузового поезда сошли с путей в Во...,4


# Предобработка с помощью UDPipe

## Установка нужных пакетов и загрузка модели

In [8]:
import wget
import os

In [19]:
def download_udpipe_model():
    #!pip install ufal.udpipe
    #!pip install wget
    udpipe_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
    udpipe_filename = udpipe_url.split('/')[-1]

    if not os.path.isfile(udpipe_filename):
        print('UDPipe model not found. Downloading...', file=sys.stderr)
        wget.download(udpipe_url)
    
    return udpipe_filename

## Тестовые тексты

In [10]:
text_url = 'https://rusvectores.org/static/henry_sobolya.txt'
text_filename = text_url.split('/')[-1]

if not os.path.isfile(text_filename):
    print('{} not found. Downloading...'.format(text_filename), file=sys.stderr)
    wget.download(text_url)

In [11]:
test_text_0 = 'Пока Ольга, Максим и Никита разговаривали по телефону 81234567890, моя мама Людмила тщательно мыла \
грязную раму картины "Последний день Владимира" \
из Третьяковки и Эрмитажа новым мылом Nivea от Яндекса и Газпрома до блеска и сверкания для передачи в \
Москву, в Валенсию, Puma, Пермь, NFL и Екатеринбург'
test_text_1 = train['text'][0]
test_text_2 = train['text'][1]
test_text_3 = open(text_filename, 'r', encoding='utf-8').read()

## Получение Conllu формата

In [12]:
from ufal.udpipe import Model, Pipeline
import unify

In [21]:
def load_udpipe_pipeline(udpipe_filename):
    model = Model.load(udpipe_filename)    
    process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')
    return model, process_pipeline

In [14]:
def clean_token(token, misc):
    """
    :param token:  токен (строка)
    :param misc:  содержимое поля "MISC" в CONLLU (строка)
    :return: очищенный токен (строка)
    """
    out_token = token.strip().replace(' ', '')
    if token == 'Файл' and 'SpaceAfter=No' in misc:
        return None
    return out_token

In [15]:
def clean_lemma(lemma, pos):
    """
    :param lemma: лемма (строка)
    :param pos: часть речи (строка)
    :return: очищенная лемма (строка)
    """
    out_lemma = lemma.strip().replace(' ', '').replace('_', '').lower()
    if '|' in out_lemma or out_lemma.endswith('.jpg') or out_lemma.endswith('.png'):
        return None
    if pos != 'PUNCT':
        if out_lemma.startswith('«') or out_lemma.startswith('»'):
            out_lemma = ''.join(out_lemma[1:])
        if out_lemma.endswith('«') or out_lemma.endswith('»'):
            out_lemma = ''.join(out_lemma[:-1])
        if out_lemma.endswith('!') or out_lemma.endswith('?') or out_lemma.endswith(',') \
                or out_lemma.endswith('.'):
            out_lemma = ''.join(out_lemma[:-1])
    return out_lemma

In [16]:
def num_replace(word):
    newtoken = 'x' * len(word)
    nw = newtoken + '_NUM'
    return nw

In [17]:
def process(pipeline, text='Строка', keep_pos=True, keep_punct=False):
    entities = {'PROPN'}
    named = False
    memory = []
    mem_case = None
    mem_number = None
    tagged_propn = []
    
    # обрабатываем текст, получаем результат в conllu
    processed = process_pipeline.process(text)

    # пропускаем строки со служебной информацией
    content = [l for l in processed.split('\n') if not l.startswith('#')]

    # извлекаем из обработанного текста леммы, тэги и морфологические характеристики
    tagged = [w.split('\t') for w in content if w]

    result = tagged

    # print(result)
    # print('='*100)

    # print(len(tagged))
    # print('='*100)

    for t in tagged:
    #     print(t, named)
        if len(t) != 10:
            continue

        (word_id, token, lemma, pos, xpos, feats, head, deprel, deps, misc) = t

        # убрали внутри токена оконечные и внутренние пробелы, перевели в нижний регистр
        token = clean_token(token, misc)
        # убрали внутри леммы оконечные и внутренние пробелы, перевели в нижний регистр, \
        # убрали оконечные знаки преминания
        lemma = clean_lemma(lemma, pos)

        if not lemma or not token:
            continue

        if pos in entities:
            # если у имени собственного нет морфологических характеристик или она только одна
            if '|' not in feats:
                tagged_propn.append('{}_{}'.format(lemma, pos))
                continue # модификация - в оригинале этого нет

            # если нет падежа или числа
            morph = {el.split('=')[0]: el.split('=')[1] for el in feats.split('|')}
            if 'Case' not in morph or 'Number' not in morph:
                tagged_propn.append('{}_{}'.format(lemma, pos))
                continue

            if not named:
                named = True
                mem_case = morph['Case']
                mem_number = morph['Number']

    #         print(token, morph, morph['Case'], mem_case, morph['Number'], mem_number, memory)

            # группируем все имена собственные одного падежа и числа в одну сущность
            if morph['Case'] == mem_case and morph['Number'] == mem_number:
                memory.append(lemma)
    #             print(lemma, memory)
                if 'SpacesAfter=\\n' in misc or 'SpacesAfter=\s\\n' in misc:
                    named = False
                    past_lemma = '::'.join(memory)
                    memory = []
                    tagged_propn.append(past_lemma + '_PROPN ')
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('{}_{}'.format(lemma, pos))

        else:
            if not named:
                # если число, то заменяем цифры на x и добавляем окончание _NUM в лемму
                if pos == 'NUM' and token.isdigit():
                    lemma = num_replace(token)
                tagged_propn.append('{}_{}'.format(lemma, pos))
            else:
                named = False
                past_lemma = '::'.join(memory)
                memory = []
                tagged_propn.append(past_lemma + '_PROPN ')
                tagged_propn.append('{}_{}'.format(lemma, pos))    

    if not keep_punct:
        tagged_propn = [word for word in tagged_propn if word.split('_')[1] != 'PUNCT']

    if not keep_pos:
        tagged_propn = [word.split('_')[0] for word in tagged_propn]

    return tagged_propn

In [22]:
udpipe_filename = download_udpipe_model()
model, process_pipeline = load_udpipe_pipeline(udpipe_filename)

line = unify.unify_sym(test_text_0)
print(line)
print('='*100)

output = process(process_pipeline, text=line, keep_pos=False, keep_punct=False)
print(output)

Пока Ольга, Максим и Никита разговаривали по телефону 81234567890, моя мама Людмила тщательно мыла грязную раму картины "Последний день Владимира" из Третьяковки и Эрмитажа новым мылом Nivea от Яндекса и Газпрома до блеска и сверкания для передачи в Москву, в Валенсию, Puma, Пермь, NFL и Екатеринбург
['пока', 'ольга', 'максим', 'и', 'никита', 'разговаривать', 'по', 'телефон', 'xxxxxxxxxxx', 'мой', 'мама', 'людмила', 'тщательно', 'мыть', 'грязный', 'рама', 'картина', 'последний', 'день', 'владимир', 'из', 'третьяковка', 'и', 'эрмитаж', 'новый', 'мыло', 'nivea', 'от', 'яндекс', 'и', 'газпром', 'до', 'блеск', 'и', 'сверкание', 'для', 'передача', 'в', 'москва', 'в', 'валенсия', 'puma', 'пермь', 'nfl', 'и', 'екатеринбург']


# Baseline

In [53]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

In [48]:
X = train.text
y = train.sentiment

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [51]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB())
              ])

In [52]:
nb.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [58]:
%%time 
y_pred = nb.predict(X_test)

CPU times: user 889 ms, sys: 4.66 ms, total: 894 ms
Wall time: 761 ms


In [59]:
print('Accuracy: {:.2f}'.format(accuracy_score(y_pred, y_test)))

Accuracy: 0.58


In [60]:
print(classification_report(y_test, y_pred, target_names=y.unique()))

              precision    recall  f1-score   support

    negative       0.79      0.03      0.07       435
    positive       0.54      0.97      0.70      1226
     neutral       0.83      0.27      0.41       818

    accuracy                           0.58      2479
   macro avg       0.72      0.43      0.39      2479
weighted avg       0.68      0.58      0.49      2479



# Gensim

In [24]:
#!pip install gensim

In [26]:
#!pip install gensim -U

In [27]:
import sys
import gensim, logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [28]:
import zipfile
model_url = 'http://vectors.nlpl.eu/repository/11/180.zip'
m = wget.download(model_url)
model_file = model_url.split('/')[-1]
with zipfile.ZipFile(model_file, 'r') as archive:
    stream = archive.open('model.bin')
    model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

2020-01-20 16:50:26,155 : INFO : loading projection weights from <zipfile.ZipExtFile name='model.bin' mode='r' compress_type=deflate>
2020-01-20 16:50:34,232 : INFO : loaded (189193, 300) matrix from <zipfile.ZipExtFile [closed]>


In [38]:
# words = ['день_NOUN', 'ночь_NOUN', 'человек_NOUN', 'семантика_NOUN', 'студент_NOUN', 'студент_ADJ']

In [39]:
# for word in words:
#     # есть ли слово в модели? Может быть, и нет
#     if word in model:
#         print(word)
#         # выдаем 10 ближайших соседей слова:
#         for i in model.most_similar(positive=[word], topn=10):
#             # слово + коэффициент косинусной близости
#             print(i[0], i[1])
#         print('\n')
#     else:
#         # Увы!
#         print(word + ' is not present in the model')

In [40]:
# print(model.similarity('человек_NOUN', 'обезьяна_NOUN'))

In [41]:
# print(model.doesnt_match('яблоко_NOUN груша_NOUN виноград_NOUN банан_NOUN лимон_NOUN картофель_NOUN'.split()))

In [42]:
# print(model.most_similar(positive=['пицца_NOUN', 'россия_NOUN'], negative=['италия_NOUN'])[0][0])

In [ ]:
#